In [1]:
import pandas as pd
import numpy as np
import requests, bs4
import re, os
import matplotlib.pyplot as plt
import statsapi
from datetime import date



In [2]:
today = date.today()
today = str(today)[-2:]
today

url = f'https://www.baseballmusings.com/cgi-bin/CurStreak.py?EndDate=07%2F{today}%2F2022'
resp = requests.get(url)

with open('test_mlb_longesthitstreak', 'wb') as f:
    f.write(resp.content)

df_streaks = pd.read_html('test_mlb_longesthitstreak')
df1 = pd.DataFrame(df_streaks[1])
df1

,Player,Games,At Bats,Runs,Hits,HR,RBI,BB,K,BA,OBA,Slug%,Last Game Date
0,Austin Riley,11,46,9,19,6,12,2,10,.413,.438,.870,2022-07-15
1,Gleyber Torres,9,40,9,16,1,5,5,9,.400,.467,.600,2022-07-15
2,Will Smith,8,31,5,14,1,9,6,6,.452,.553,.710,2022-07-15
3,Brandon Drury,8,34,6,13,1,6,3,7,.382,.421,.647,2022-07-15
4,Corey Seager,8,28,8,11,6,12,7,3,.393,.486,1.071,2022-07-15
5,Francisco Lindor,8,36,4,10,3,8,1,6,.278,.297,.611,2022-07-14
6,Carson Kelly,8,27,4,10,1,3,1,4,.370,.393,.556,2022-07-15
7,Freddie Freeman,7,28,6,16,2,8,3,1,.571,.606,.964,2022-07-15
8,Teoscar Hernandez,7,27,5,11,3,10,2,10,.407,.448,.852,2022-07-15
9,Andrew McCutchen,7,30,5,10,2,9,1,10,.333,.344,.567,2022-07-15


In [3]:
name = 'Bo Bichette'
x = df1.loc[(df1['Player']== 'Bo ')]
type(x)

pandas.core.frame.DataFrame

In [4]:
'Juan Soto' in df1['Player'].value_counts()

True